In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os , sys ,cv2
import json
import time
import csv

import keras.applications
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model


from urllib.request import urlopen

from PIL import Image

from sklearn.preprocessing import StandardScaler
from sklearn import svm

import shutil

In [ ]:
# Create new folder to store the snapshots from train videos
try:
    os.mkdir("./output_dir")
except FileExistsError:
    pass
# Create new folder to store the snapshots from test videos
try:
    os.mkdir("./test_output_dir")
except FileExistsError:
    pass

In [ ]:
# Get the snapshots of the training videos for 1/3rd frame of the total length
pathOut = "./output_dir/"
listing = os.listdir("./train_video/")
count = 0
counter = 1

for vid in listing:
    vid1 = "./train_video/"+vid
    cap = cv2.VideoCapture(vid1)
    frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / 3) # Divide by 3 to get correct frame
    count = 0
    counter += 1
    success = True
    while success:
        success,image = cap.read()
        print('read a new frame:',success)
        if count == frame:
            cv2.imwrite(pathOut + str(vid) + ".jpg",image)
            print (success)
        count+=1

In [ ]:
# Get the snapshots of the test videos for 1/3rd frame of the total length
pathOut = "./test_output_dir/"
listing = os.listdir("./test_video/")
count = 0
counter = 1

for vid in listing:
    vid1 = "./test_video/"+vid
    cap = cv2.VideoCapture(vid1)
    frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) / 3) # Divide by 3 to get proper frame
    count = 0
    counter += 1
    success = True
    while success:
        success,image = cap.read()
        print('read a new frame:',success)
        if count == frame:
            cv2.imwrite(pathOut + str(vid) + ".jpg",image)
            print (success)
        count+=1

In [ ]:
# Read the labels for the training images after appending .jpg to the image filename
df_train = pd.read_csv('tarin_tag.txt',header=None, sep=',')
df_train.columns = ['filename','label']
df_train['filename'] = df_train['filename'] + '.jpg'
df_train.head(5)

In [ ]:
filename = df_train.filename.values
label = df_train.label.values

In [ ]:
# Function to get the image size in the image directory
def file_size_train(file_name):
    stats=os.stat("output_dir/"+str(file_name))
    return stats.st_size

In [ ]:
# Function to call the keras pre-trained model to get feature weights of the train images
def resnet_model_train():
    n=0
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    images_path = "./output_dir"
    # Looping over every image present in the files list
    for img_path in filename:
        if(file_size_train(img_path)!=0):
            print(str(img_path))
            n+=1
            print(n)
            # load the image and resize it
            img = image.load_img("./output_dir/"+str(img_path), target_size=(224, 224))
            # extract features from each image
            x_image = image.img_to_array(img)
            x_image = np.expand_dims(x_image, axis=0) # increase dimensions of x to make it suitable for further feature extraction
            x_image = preprocess_input(x_image)
            x_features = model.predict(x_image) # extract image features from model
            x_features = np.array(x_features) # convert features list to numpy array
            x_flatten= x_features.flatten() # flatten out the features in x
            train_features.append(x_flatten) # this list contains the final features of the train images

In [ ]:
# Function to get the image size in the image directory
def file_size_test(file_name):
    stats=os.stat("test_output_dir/"+str(file_name))
    return stats.st_size

In [ ]:
# Function to call the keras pre-trained model to get feature weights of the test images
def resnet_model_test():
    n=0
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    images_path = "./test_output_dir"
    for f in os.listdir(images_path):
        test_files.append(f)
    # Looping over every image present in the files list
    for img_path in test_files:
        if(file_size_test(img_path)!=0):
            print(str(img_path))
            n+=1
            print(n)
            # load the image and resize it
            img = image.load_img("./test_output_dir/"+str(img_path), target_size=(224, 224))
            # extract features from each image
            x_image = image.img_to_array(img)
            x_image = np.expand_dims(x_image, axis=0) # increase dimensions of x to make it suitable for further feature extraction
            x_image = preprocess_input(x_image)
            x_features = model.predict(x_image) # extract image features from model
            x_features = np.array(x_features) # convert features list to numpy array
            x_flatten= x_features.flatten() # flatten out the features in x
            test_features.append(x_flatten) # this list contains the final features of the test images

In [ ]:
# Get features for every image extracted from train videos
train_features=[]
resnet_model_train()

In [ ]:
# Scale all features with standard scaler
ss = StandardScaler()
train_features = ss.fit_transform(train_features)

In [ ]:
# Get features for every image extracted from test videos
test_features=[]
test_files = []
resnet_model_test()
test_features = ss.transform(test_features) # scale the test video frames

In [ ]:
#Train the model with SVC on extracted features and provided labels
model = svm.SVC()
model.fit(train_features,label)
pred = model.predict(test_features) # predict the labels for test videos

In [ ]:
# Strip all .jpg from the end of the filename
test_files1 = [s.strip('.jpg') for s in test_files]

In [ ]:
# Write the output in CSV format
label_output = pd.DataFrame(pred, columns=['label'])
testfile_output = pd.DataFrame(test_files1, columns=['file_name'])

In [ ]:
result = pd.concat([testfile_output, label_output], axis=1, ignore_index = True)

In [ ]:
result.columns=['file_name','label']
result.head(5)

In [ ]:
result.to_csv('output.csv', header=True, index=False) # save output in csv